In [1]:
import pinecone
from dotenv import load_dotenv
import os
import pandas as pd
import torch
def configure():
    load_dotenv()

In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
#from langchain.lims import CTransformers
from transformers import AutoModelForCausalLM, AutoTokenizer

#Database
import chromadb

e:\conda\ANACONDA\envs\cuda_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print("CUDA Available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

CUDA Available: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU


API KEY LOADERS

In [4]:
PINECONE_API_KEY = os.getenv('api_key')
PINECONE_API_ENV = os.getenv('env_name')

LOADING DATA

In [5]:
def load_data(path):
    df = pd.read_csv(path)
    df['text'] = 'Question/n' + df['question'] + 'Answer/n' + df['answer']
    print(df.head())
    return df
 
dataset = load_data(path = "./data/cleaned_file.csv")

                                 question  \
0                What is (are) Glaucoma ?   
1                  What causes Glaucoma ?   
2     What are the symptoms of Glaucoma ?   
3  What are the treatments for Glaucoma ?   
4                What is (are) Glaucoma ?   

                                              answer  \
0  Glaucoma is a group of diseases that can damag...   
1  Nearly 2.7 million people have glaucoma, a lea...   
2  Symptoms of Glaucoma  Glaucoma can develop in ...   
3  Although open-angle glaucoma cannot be cured, ...   
4  Glaucoma is a group of diseases that can damag...   

                                                text  
0  Question/nWhat is (are) Glaucoma ?Answer/nGlau...  
1  Question/nWhat causes Glaucoma ?Answer/nNearly...  
2  Question/nWhat are the symptoms of Glaucoma ?A...  
3  Question/nWhat are the treatments for Glaucoma...  
4  Question/nWhat is (are) Glaucoma ?Answer/nGlau...  


Split the data into Chunks

In [ ]:
def text_splitter(extacted_data):
    documents = [Document(page_content=text) for text in extacted_data]
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks 

In [7]:
list_ = dataset['text'].tolist()
text_chunks = text_splitter(list_)
texts = [chunk.page_content for chunk in text_chunks]
#text_chunks = text_splitter()

In [12]:
#to know the chunk size
print("Length of the chunk size", len(text_chunks))

Length of the chunk size 58594


EMBEDDINGS MODEL DOWNLOAD

In [13]:
#Download the Embeddings from hugging face
####
#def download_huggingface_embedding():
 #   embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
 #   return embeddings
####

In [14]:
###embeddings = download_huggingface_embedding()###

CONVERTING THE TEXT TO VECTOR

In [15]:
# Convert chunks to vector embeddings
# Load the embedding model on GPU
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

# Convert chunks to vector embeddings using GPU
embeddings = embedding_model.encode(texts, convert_to_tensor=True, device="cuda")

CHROMADB SETUP

In [ ]:
#Now we are gonna setup our CHROMA VECTOR DB 
"""
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="medical_chatbot")#creating the collection

for i, text in enumerate(texts):
    collection.add(
        ids=[str(i)],  # Unique ID for each document
        documents=[text],  # Raw text
        embeddings=[embeddings[i].tolist()],  # Convert NumPy array to list
    )

print("Documents stored in ChromaDB successfully!")"""

Insert of existing embedding ID: 0
Add of existing embedding ID: 0
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 3
Add of existing embedding ID: 3
Insert of existing embedding ID: 4
Add of existing embedding ID: 4
Insert of existing embedding ID: 5
Add of existing embedding ID: 5
Insert of existing embedding ID: 6
Add of existing embedding ID: 6
Insert of existing embedding ID: 7
Add of existing embedding ID: 7
Insert of existing embedding ID: 8
Add of existing embedding ID: 8
Insert of existing embedding ID: 9
Add of existing embedding ID: 9
Insert of existing embedding ID: 10
Add of existing embedding ID: 10
Insert of existing embedding ID: 11
Add of existing embedding ID: 11
Insert of existing embedding ID: 12
Add of existing embedding ID: 12
Insert of existing embedding ID: 13
Add of existing embedding ID: 13
Insert of existing embedding ID: 14
Add of existing em

Documents stored in ChromaDB successfully!


Retrive the Relevant Chunks from the ChromaDB

In [18]:
def retrieve_relevant_chunks(query, top_k=3,):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True, device="cuda").tolist()

    # Perform similarity search
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    retrieved_texts = results["documents"][0]  # Top-k documents
    return retrieved_texts

# Example Query
query = "What are the symptoms of diabetes?"
retrieved_texts = retrieve_relevant_chunks(query)
print("Relevant Chunks:", retrieved_texts)

Relevant Chunks: ['Question/nWhat are the symptoms of Diabetes ?Answer/nMany people with diabetes experience one or more symptoms, including extreme thirst or hunger, a frequent need to urinate and/or fatigue. Some lose weight without trying. Additional signs include sores that heal slowly, dry, itchy skin, loss of feeling or tingling in the feet and blurry eyesight. Some people with diabetes, however, have no symptoms at all.', 'urination  - feeling very hungry or tired  - losing weight without trying  - having sores that heal slowly  - having dry, itchy skin  - loss of feeling or tingling in the feet  - having blurry eyesight. being very thirsty frequent urination feeling very hungry or tired losing weight without trying having sores that heal slowly having dry, itchy skin loss of feeling or tingling in the feet having blurry eyesight. Signs of type 1 diabetes usually develop over a short period of time. The signs for', 'Question/nWhat are the symptoms of Your Guide to Diabetes: Type

Loading the LLAMA 2 (7B)

In [27]:
gpu_mem = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # Convert bytes to GB
gpu_layers = min(50, int(gpu_mem * 5))  # Adjust dynamically

In [ ]:
# Path to your locally saved LLAMA 2 model
local_model_path = "E:/Project/LLM_HEALTH_LOCAL_DATA/Model/Model/"

from ctransformers import AutoModelForCausalLM

# Load GGUF model
model = AutoModelForCausalLM.from_pretrained(
    "E:/Project/LLM_HEALTH_LOCAL_DATA/Model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    gpu_layers=gpu_layers  # Adjust based on GPU memory
)

# Generate text
response = model("What are the symptoms of diabetes?", max_new_tokens=200)
print(response)


 Unterscheidung between type 1 and type 2 diabetes can be difficult as there is often a blurring of the lines between the two conditions. However, here are some common symptoms of each type of diabetes:
Type 1 Diabetes Symptoms:
* Frequent urination
* Increased thirst
* Fatigue
* Blurred vision
* Slow healing of cuts and wounds
* Tingling or numbness in the hands and feet
* Recurring skin, gum, or bladder infections
* Flu-like symptoms (due to hypoglycemia)
Type 2 Diabetes Symptoms:
* Increased thirst
* Frequent urination
* Fatigue
* Blurred vision
* Cuts or wounds that are slow to heal
* Tingling or numbness


RAG IMPLEMENTATION

In [6]:
# Function to retrieve relevant documents and generate a response
def rag_pipeline(query):
    # Step 1: Encode the query
    query_embedding = embedding_model.encode(query).tolist()
    
    # Step 2: Retrieve relevant documents from ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=2  # Retrieve top 2 matching documents
    )

    retrieved_docs = results["documents"][0] if results["documents"] else []
    
    # Step 3: Prepare context for LLAMA 2
    if retrieved_docs:
        context = "\n".join(retrieved_docs)
    else:
        context = "No relevant documents were found in the database."

    # Step 4: Generate response using LLAMA 2
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    
    # ✅ FIX: Use `model(prompt)` instead of `model.generate()`
    response = model(prompt, max_new_tokens=200)  # Direct function call

    return response

# Test RAG pipeline
query = "I have dizziness?"
response = rag_pipeline(query)
print("RAG Response:", response)


NameError: name 'embedding_model' is not defined